In [18]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display
from api_clients import create_clients
from system_info import retrieve_system_info
import gradio as gr

c:\Users\paing\anaconda3\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
clients = create_clients()
system_info = retrieve_system_info()


In [3]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

response = clients["groq"].chat.completions.create(
    model=clients["models"]["GROQ_MODEL"], 
    messages=[{
      "role": "user", "content": message
    }]
    )
display(Markdown(response.choices[0].message.content))

### Short answer

**You already have a compiler** (`g++ 6.3.0` from MinGW) so you do **not** need to install anything extra to compile a single file.  
If you want the *fastest* possible run‑time you can (and should) upgrade to a newer MinGW‑w64 toolchain (e.g. 11.x or 12.x), but the existing one will still compile and run your code fine.

Below are two sets of instructions:

| What you want | What you need |
|---------------|---------------|
| **Compile a single file with the existing compiler** | `g++ -O3 -march=native -std=c++17 -o main.exe main.cpp` |
| **Compile and run it from a Python script** | `['g++', '-O3', '-march=native', '-std=c++17', '-o', 'main.exe', 'main.cpp']` & `['main.exe']` |

---

## 1. Using the existing compiler (command‑line)

```cmd
g++ -O3 -march=native -std=c++17 -o main.exe main.cpp
main.exe
```

- `-O3` : aggressive optimisation (most common for “fastest” code).  
- `-march=native` : enable CPU‑specific instructions for your Ryzen 7.  
- `-std=c++17` : modern C++ (you can bump to c++20/23 if your code uses it).  
- `-o main.exe` : name the executable `main.exe` (Windows convention).  

You can add `-flto` (link‑time optimisation) and `-s` (strip symbols) for a smaller binary if you like.

---

## 2. Using the existing compiler from **Python**

```python
import subprocess

# --- Compile -------------------------------------------------------------
compile_command = [
    "g++",            # compiler
    "-O3",            # optimisation level
    "-march=native",  # use CPU‑specific instructions
    "-std=c++17",     # C++ standard
    "-o", "main.exe", # output binary
    "main.cpp"        # source file
]

compile_result = subprocess.run(
    compile_command,
    check=True,
    text=True,
    capture_output=True
)

# --- Run ---------------------------------------------------------------
run_command = [
    "main.exe"        # executable (Windows)
]

run_result = subprocess.run(
    run_command,
    check=True,
    text=True,
    capture_output=True
)

print(run_result.stdout)
```

### Why this works

* `subprocess.run(..., check=True)` will raise a `CalledProcessError` if the compiler or program exits with a non‑zero status – useful for debugging.  
* `capture_output=True` collects the standard output so you can inspect it later.  
* Passing the command as a *list* (rather than a single string) lets Python avoid any shell‑escaping headaches, which is especially important on Windows.

---

## 3. Optional: Upgrade to a newer MinGW‑w64 (if you want better performance)

If you’d like the latest compiler features or better optimisation, follow these quick steps:

1. **Uninstall** the old MinGW‑w64 (if you wish).  
2. Install **MSYS2**: <https://www.msys2.org/>  
3. Open the *MSYS2 MinGW 64‑bit* terminal and run:

   ```bash
   pacman -Syu            # update package database
   pacman -S mingw-w64-x86_64-toolchain   # install GCC, g++, etc.
   ```

4. Add MSYS2’s `mingw64\bin` directory to your system `PATH` (or just use the MSYS2 terminal for compiling).  
5. Verify:

   ```cmd
   g++ --version
   ```

   You should see something like `g++ (x86_64-w64-mingw32) 11.3.0`.

After that, the **same compile/run commands** above will work and will give you noticeably better optimisation on modern CPUs.

---

### TL;DR

- **No installation needed** – you already have `g++` available.  
- Compile & run with:

  ```python
  compile_command = ["g++","-O3","-march=native","-std=c++17","-o","main.exe","main.cpp"]
  run_command     = ["main.exe"]
  ```

- If you want the absolute best performance, upgrade to a newer MinGW‑w64 via MSYS2 and then keep the same commands.

In [ ]:
compile_command = ["g++","-std=c++17","-O3","-march=native","-ffast-math","-funroll-loops","-o", "main.exe","main.cpp"]
run_command = ["./main.exe"]

In [5]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
      Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
      The system information is:
      {system_info}
      Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
      {compile_command}
      Respond only with C++ code.
      Python code to port:

      ```python
      {python}
      ```
    """

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [8]:
def port(client, model, python):
    reasoning_effort = "high" if 'groq' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [15]:
def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [9]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [10]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [11]:
run_python(pi)

Result: 3.141592656089
Execution Time: 54.295726 seconds


In [13]:
port(clients["groq"], clients["models"]["GROQ_MODEL"], pi)

In [16]:
compile_and_run()

Result: 3.141592656455
Execution Time: 0.383051 seconds

Result: 3.141592656455
Execution Time: 0.379768 seconds

Result: 3.141592656455
Execution Time: 0.385732 seconds



In [44]:
import io
import sys

def execute_python_code(code):
  try:
    output = io.StringIO()
    sys.stdout = output
    exec(code)
  finally:
    sys.stdout = sys.__stdout__
  return output.getvalue()

def execute_cpp_code(code):
  try:
    compile_command = ["g++","-std=c++17","-O3","-march=native","-ffast-math","-funroll-loops","-o", "main.exe","main.cpp"]
    run_command = ["./main.exe"]
    compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
    run_result = subprocess.run(run_command, check=True, text=True, capture_output=True).stdout
    return run_result 
  except subprocess.CalledProcessError as e:
    return f"An error occurred:\n{e.stderr}"

In [45]:
def stream_groq(python):
  result = clients["groq"].chat.completions.create(
    model=clients["models"]["GROQ_MODEL"], 
    messages=[
      {
        "role": "user", "content": user_prompt_for(python)
      }
    ], 
    stream=True
  )
  reply = ""
  with result as s:
    for event in s:
        delta = event.choices[0].delta.content
        if delta:
            reply += delta
            yield reply.replace('```cpp\n', '').replace('```', '')

In [46]:
def optimize(python, model):
  if model == 'Groq':
    result = stream_groq(python)
  else:
    raise ValueError("Unknown model ", model)
  for stream in result:
    yield stream

In [49]:
with gr.Blocks() as ui:
  gr.Markdown("Convert Python to C++")
  with gr.Row():
    python = gr.Textbox(label="Python", lines=10, value=pi)
    cpp = gr.Textbox(label="C++", lines=10)
  with gr.Row():
    model = gr.Dropdown(["Groq"], label="Select model", value="Groq")
  with gr.Row():
    convert  = gr.Button("Convert code")
  with gr.Row():
    python_run = gr.Button("Run Python")
    cpp_run = gr.Button("Run C++")
  with gr.Row():
    python_output = gr.TextArea(label="Python Result:", elem_classes=["python"])
    cpp_output = gr.TextArea(label="C++ Result:", elem_classes=["cpp"])
  
  convert.click(optimize, inputs=[python, model], outputs=[cpp])
  python_run.click(execute_python_code, inputs=[python], outputs=[python_output])
  cpp_run.click(execute_cpp_code, inputs=[cpp], outputs=[cpp_output])
ui.launch()